In [1]:
import numpy as np
from utils.database_repository import DataRepository

In [2]:
repository = DataRepository()

ERROR:root:

-------
ERROR:
    User secrets are used in this project, but the file has not been made locally.
    Please create the file '/secrets/appsettings.secrets.json' and input all required secrets and try agian.
------

NoneType: None


c:\secrets\appsettings.secrets.json
I am unable to connect to the database
exceptions must derive from BaseException


In [3]:
test_case = "DiningPhilosophers"
dut = "Surface4Pro"
profiler = "IntelPowerGadget"

min_temp = 0
max_temp = 200

min_battery = 1
max_battery = 100

minutes_between = 0
duration = 2

version = 2

In [3]:
repository.query("SELECT * FROM RawData")

[(1,
  3,
  '{"SystemTime":"12:55:48.1300000","RDTSC":25630114475810,"ElapsedTimesec":58.408,"CPUUtilization":18,"CPUFrequency_0MHz":1500,"ProcessorPower_0Watt":2.094,"CumulativeProcessorEnergy_0Joules":122.321,"CumulativeProcessorEnergy_0mWh":33.978,"IAPower_0Watt":1.271,"CumulativeIAEnergy_0Joules":74.212,"CumulativeIAEnergy_0mWh":20.614,"PackageTemperature_0C":42,"PackageHot_0":0,"DRAMPower_0Watt":0.84,"CumulativeDRAMEnergy_0Joules":49.064,"CumulativeDRAMEnergy_0mWh":13.629,"GTPower_0Watt":0.017,"CumulativeGTEnergy_0Joules":1.017,"CumulativeGTEnergy_0mWh":0.283,"PackagePL1_0Watt":15,"PackagePL2_0Watt":44,"PackagePL4_0Watt":44,"PlatformPsysPL1_0Watt":0,"PlatformPsysPL2_0Watt":0,"GTFrequencyMHz":7,"GTUtilization":44.945,"TotalElapsedTimeInSeconds":58.40817642211914,"MeasuredRdtscFrequencyInGhz":1.7999999523162842,"AverageProcessorPower0InWatt":2.094248056411743,"AverageIAPower0InWatt":1.2705769538879395,"AverageDRAMPower0InWatt":0.840025007724762,"AverageGTPower0InWatt":0.017412999644